In [1]:
var('t q')

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition)-i-1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition)-number_of_ones(partition)

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t+t^my_len(list_of_list_of_partition[i][j])
        series = series+series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail)-1:
        return False
    partition2 = partition[len(partition)-len(tail)+1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition)-len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition)-len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def vector_sum(vector1, vector2):
    return {partition: vector1.get(partition, 0)+vector2.get(partition, 0)
            for partition in set(vector1)|set(vector2)}

def vector_simplify0(partition):
    for i in range(len(partition)-1):
        if partition[i] < partition[i+1]:
            return {partition[0: i]+(partition[i]+partition[i+1],)+partition[i+2: len(partition)]:
                    partition[i+1]-partition[i], partition[0: i]+(partition[i+1],)+(partition[i],)
                    +partition[i+2: len(partition)]: 1}
    return {partition: 1}

def vector_simplify(vector):
    vector2 = {}
    for partition in set(vector):
        vector3 = vector_simplify0(partition)
        vector2 = vector_sum(vector2, {partition2:vector3[partition2]*vector[partition]
                                       for partition2 in set(vector3)})
    if vector2 == vector:
        return vector2
    return vector_simplify(vector2)

def L_partition_vector(partition, vector):
    return vector_simplify(
        {partition+partition2: vector[partition2] for partition2 in set(vector)})

def my_order(a):
    polynomial = to_polynomial(a)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(a), len(a)-number_of_ones(a), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition)-len(partition2)+1):
        if partition[i: i+len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [3, 3], [4, 3, 1], [4, 4, 1], [6, 5, 3, 1], [3, 1, 1], [5, 4, 1, 1], [1, 1, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition)-2):
            if partition[i]-partition[i+2] <= 2:
                return True
    if len(partition) > 3:
        for i in range(len(partition)-3):
            if (partition[i]-partition[i+3] <= 3
                or (partition[i]-partition[i+3] <= 4 and partition[i+1]-partition[i+2] <= 3)):
                return True
    if len(partition) > 4:
        for i in range(len(partition)-4):
            if (partition[i]-partition[i+1] == 1 and partition[i+1]-partition[i+2] == 2
                and partition[i+2]-partition[i+3] == 2 and partition[i+3]-partition[i+4] == 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    m2 = {(1, 1): 3, (2,): -4}
    m3 = {(1, 1, 1):4, (2, 1): -12, (3,): 3}
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n-2)
    list_of_partition3 = ordered_partitions(n-3)
    list_of_vector_m2 = [L_partition_vector(tuple(list_of_partition2[i]), m2)
                         for i in range(len(list_of_partition2))]
    list_of_vector_m3 = [L_partition_vector(tuple(list_of_partition3[i]), m3)
                         for i in range(len(list_of_partition3))]
    matrix_m2 = matrix({(i, j): list_of_vector_m2[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({(i, j): list_of_vector_m3[i].get(tuple(list_of_partition[j]), 0)
                        for j in range(len(list_of_partition))
                        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def to_polynomial(partition):
    polynomial = [0]*(sum(partition)-1)
    for i in range(len(partition)-number_of_ones(partition)):
        polynomial[partition[i]-2] += 1
    return polynomial

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n+1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n+1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n+1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [2]:
ordered_partitions(15)

[[2, 2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2, 1, 1, 1],
 [2, 2, 2, 2, 2, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 2, 2, 2, 2],
 [3, 2, 2, 2, 2, 2, 1, 1],
 [3, 2, 2, 2, 2, 1, 1, 1, 1],
 [3, 2, 2, 2, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 2, 2, 2, 2, 1],
 [4, 2, 2, 2, 2, 2, 1],
 [3, 3, 2, 2, 2, 1, 1, 1],
 [4, 2, 2, 2, 2, 1, 1, 1],
 [3, 3, 2, 2, 1, 1, 1, 1, 1],
 [4, 2, 2, 2, 1, 1, 1, 1, 1],
 [3, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 3, 2, 2, 2],
 [4, 3, 2, 2, 2, 2],
 [5, 2, 2, 2, 2, 2],
 [3, 3, 3, 2, 2, 1, 1],
 [4, 3, 2, 2, 2, 1, 1],
 [5, 2, 2, 2, 2, 1,

In [3]:
basis_up_tov2(15)==basis_up_to(15)

True

In [4]:
basis_up_to(15)

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [6, 3, 1], [5, 5], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [7, 3, 1],
  [6, 5],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [7, 4, 1],
  [8, 3, 1],
  [6, 6],
  [7, 5],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [7, 4, 1, 1],
  [6, 6, 1],
  [7, 5, 1],
  [8, 4, 1],
  [9, 3, 1],
  [7, 6],
  [8, 5],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [7, 4, 3],
  [6, 6, 1, 1],
  [7, 5, 1, 1],
  [8, 4, 1, 1],
  [7, 6, 1],
  [8, 5, 1],
  [9, 4, 1],
  [10, 3, 1],
  [7, 7],
  [8, 6],
  [9, 5],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [8, 4, 3],
  [7, 6, 1, 1],
 

In [5]:
pivots_up_to(15)

[[],
 [],
 [[2]],
 [[2, 1], [1, 1, 1]],
 [[2, 2], [2, 1, 1], [1, 1, 1, 1]],
 [[2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1], [3, 2], [3, 1, 1]],
 [[2, 2, 2],
  [2, 2, 1, 1],
  [2, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [3, 2, 1],
  [3, 1, 1, 1],
  [3, 3],
  [4, 2]],
 [[2, 2, 2, 1],
  [2, 2, 1, 1, 1],
  [2, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2],
  [3, 2, 1, 1],
  [3, 1, 1, 1, 1],
  [3, 3, 1],
  [4, 2, 1],
  [4, 1, 1, 1],
  [5, 2]],
 [[2, 2, 2, 2],
  [2, 2, 2, 1, 1],
  [2, 2, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 1],
  [3, 2, 1, 1, 1],
  [3, 1, 1, 1, 1, 1],
  [3, 3, 2],
  [4, 2, 2],
  [3, 3, 1, 1],
  [4, 2, 1, 1],
  [4, 1, 1, 1, 1],
  [4, 3, 1],
  [5, 2, 1],
  [5, 1, 1, 1],
  [6, 2]],
 [[2, 2, 2, 2, 1],
  [2, 2, 2, 1, 1, 1],
  [2, 2, 1, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 2],
  [3, 2, 2, 1, 1],
  [3, 2, 1, 1, 1, 1],
  [3, 1, 1, 1, 1, 1, 1],
  [3, 3, 2, 1],
  [4, 2, 2, 1],
  [3, 3, 1, 1, 1],
  

In [6]:
series_t_q(basis_up_to(20))

(3*t^8 + 9*t^7 + 19*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (t^8 + 7*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (5*t^7 + 14*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (3*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (2*t^7 + 9*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (5*t^6 + 4*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (3*t^6 + 4*t^5 + 5*t^4 + t^3 + t^2)*q^13 + (2*t^6 + 3*t^5 + 5*t^4 + t^3 + t^2)*q^12 + (3*t^5 + 4*t^4 + t^3 + t^2)*q^11 + (2*t^5 + 4*t^4 + t^3 + t^2)*q^10 + (t^5 + 3*t^4 + t^3 + t^2)*q^9 + (3*t^4 + t^3 + t^2)*q^8 + (2*t^4 + t^3 + t^2)*q^7 + (t^4 + t^3 + t^2)*q^6 + (t^3 + t^2)*q^5 + (t^3 + t^2)*q^4 + q^3*t^2 + q^2*t^2 + q*t + 1

In [7]:
series_t_q(list_of_list_of_partition_ends(basis_up_to(20), [5,4], True))

(3*t^6 + t^4)*q^20 + (2*t^6 + t^4)*q^19 + q^16*t^4 + (2*t^6 + t^4)*q^18 + q^15*t^4 + (t^6 + t^4)*q^17 + q^14*t^4 + q^13*t^4 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^4*t^2